<a href="https://colab.research.google.com/github/kiransoorya/Prodigy_Infotech/blob/PRODIGY_GA_01/Train_a_GPT_2_Text_Generating_Model_w_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This code is a replica of Max Woolf's code.

This code is to generate text using previous text file data using prebuilt model .

The model used here is GPT-2

Make sure to run in a GPU environment else it takes a lot of time to process

Ensure you have a text file in your main drive (my drive) and not else where for proper functioning of this code.




To get started:

1. Copy this notebook to your Google Drive to keep it and save your changes. (File -> Save a Copy in Drive)
2. Make sure you're running the notebook in Google Chrome.
3. Run the cells below:


In [ ]:
!pip install -q gpt-2-simple
!pip uninstall tensorflow
!pip install tensorflow==2.8.0 #The later versions of this library has extra modules which can cause conflict
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files

  Preparing metadata (setup.py) ... done
Found existing installation: tensorflow 2.15.0
Uninstalling tensorflow-2.15.0:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/import_pb_to_tensorboard
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.10/dist-packages/tensorflow-2.15.0.dist-info/*
    /usr/local/lib/python3.10/dist-packages/tensorflow/*
Proceed (Y/n)? y
  Successfully uninstalled tensorflow-2.15.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 497.6/497.6 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 77.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.5/462.5 kB 45.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 76.4 MB/s

In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()  # Disable TensorFlow 2.x behaviors
tf.reset_default_graph()


Instructions for updating:
non-resource variables are not supported in the long term


## GPU

Colaboratory uses either a Nvidia T4 GPU or an Nvidia K80 GPU. The T4 is slightly faster than the old K80 for training GPT-2, and has more memory allowing you to train the larger GPT-2 models and generate more text.

You can verify which GPU is active by running the cell below.

In [ ]:
!nvidia-smi

Tue Jun 18 16:22:50 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## Downloading GPT-2

If you're retraining a model on new text, you need to download the GPT-2 model first.

There are three released sizes of GPT-2:

* `124M` (default): the "small" model, 500MB on disk.
* `355M`: the "medium" model, 1.5GB on disk.
* `774M`: the "large" model, cannot currently be finetuned with Colaboratory but can be used to generate text from the pretrained model (see later in Notebook)
* `1558M`: the "extra large", true model. Will not work if a K80/P4 GPU is attached to the notebook. (like `774M`, it cannot be finetuned).

Larger models have more knowledge, but take longer to finetune and longer to generate text. You can specify which base model to use by changing `model_name` in the cells below.

The next cell downloads it from Google Cloud Storage and saves it in the Colaboratory VM at `/models/<model_name>`.

This model isn't permanently saved in the Colaboratory VM; you'll have to redownload it if you want to retrain it at a later time.

In [ ]:
gpt2.download_gpt2(model_name="124M")

Fetching checkpoint: 1.05Mit [00:00, 769Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 2.00Mit/s]
Fetching hparams.json: 1.05Mit [00:00, 775Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 498Mit [00:19, 25.8Mit/s]                                  
Fetching model.ckpt.index: 1.05Mit [00:00, 840Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 3.09Mit/s]
Fetching vocab.bpe: 1.05Mit [00:00, 2.74Mit/s]


## Mounting Google Drive

The best way to get input text to-be-trained into the Colaboratory VM, and to get the trained model *out* of Colaboratory, is to route it through Google Drive *first*.

Running this cell (which will only work in Colaboratory) will mount your personal Google Drive in the VM, which later cells can use to get data in/out. (it will ask for an auth code; that auth is not saved anywhere)

In [ ]:
gpt2.mount_gdrive()

Mounted at /content/drive


## Uploading a Text File to be Trained to Colaboratory

In the Colaboratory Notebook sidebar on the left of the screen, select *Files*. From there you can upload files:

![alt text](https://i.imgur.com/TGcZT4h.png)

Upload **any smaller text file**  (<10 MB) and update the file name in the cell below, then run the cell.

In [ ]:
file_name = "shakespeare.txt"

If your text file is larger than 10MB, it is recommended to upload that file to Google Drive first, then copy that file from Google Drive to the Colaboratory VM.

In [ ]:
gpt2.copy_file_from_gdrive(file_name)

## Finetune GPT-2

The next cell will start the actual finetuning of GPT-2. It creates a persistent TensorFlow session which stores the training config, then runs the training for the specified number of `steps`. (to have the finetuning run indefinitely, set `steps = -1`)

The model checkpoints will be saved in `/checkpoint/run1` by default. The checkpoints are saved every 500 steps (can be changed) and when the cell is stopped.

The training might time out after 4ish hours; make sure you end training and save the results so you don't lose them!

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

Other optional-but-helpful parameters for `gpt2.finetune`:


*  **`restore_from`**: Set to `fresh` to start training from the base GPT-2, or set to `latest` to restart training from an existing checkpoint.
* **`sample_every`**: Number of steps to print example output
* **`print_every`**: Number of steps to print training progress.
* **`learning_rate`**:  Learning rate for the training. (default `1e-4`, can lower to `1e-5` if you have <1MB input data)
*  **`run_name`**: subfolder within `checkpoint` to save the model. This is useful if you want to work with multiple models (will also need to specify  `run_name` when loading the model)
* **`overwrite`**: Set to `True` if you want to continue finetuning an existing model (w/ `restore_from='latest'`) without creating duplicate copies.

In [ ]:

# Start a new TensorFlow session
sess = gpt2.start_tf_sess()

# Fine-tune the GPT-2 model
gpt2.finetune(sess,
              dataset= file_name,  # Replace 'file_name' with your dataset file path
              model_name='124M',
              steps=1000,
              restore_from='latest',
              run_name='run1',
              print_every=10,
              sample_every=200,
              save_every=500,
              overwrite=True
              )


ValueError: Variable model/wpe already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "/usr/local/lib/python3.10/dist-packages/gpt_2_simple/src/model.py", line 188, in model
    wpe = tf.compat.v1.get_variable('wpe', [hparams.n_ctx, hparams.n_embd],
  File "/usr/local/lib/python3.10/dist-packages/gpt_2_simple/gpt_2.py", line 198, in finetune
    output = model.model(hparams=hparams, X=context, gpus=gpus, reuse=reuse)
  File "<ipython-input-8-677d9d0b3f74>", line 5, in <cell line: 5>
    gpt2.finetune(sess,
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3473, in run_ast_nodes
    if (await self.run_code(code, result,  async_=asy)):


After the model is trained, you can copy the checkpoint folder to your own Google Drive.

If you want to download it to your personal computer, it's strongly recommended you copy it there first, then download from Google Drive. The checkpoint folder is copied as a `.rar` compressed file; you can download it and uncompress it locally.

In [ ]:
gpt2.copy_checkpoint_to_gdrive(run_name='run1')

You're done! Feel free to go to the **Generate Text From The Trained Model** section to generate text based on your retrained model.

## Load a Trained Model Checkpoint

Running the next cell will copy the `.rar` checkpoint file from your Google Drive into the Colaboratory VM.

In [ ]:
gpt2.copy_checkpoint_from_gdrive(run_name='run1')

The next cell will allow you to load the retrained model checkpoint + metadata necessary to generate text.

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

In [ ]:
import os
os.kill(os.getpid(), 9) # restart VM

In [ ]:
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, run_name='run1')

Loading checkpoint checkpoint/run1/model-1


## Generate Text From The Trained Model

After you've trained the model or loaded a retrained model from checkpoint, you can now generate text. `generate` generates a single text from the loaded model.

In [ ]:
gpt2.generate(sess, run_name='run1')

                                                Exeunt




SCENE IV.
A street. Enter Menteith and Ganymede, and Banquo.

  MENTEITH. From the man that kept the Jew from the King, came to this place the Jew, his attorney, and his lieutenant.

  GANYMEDE. 'Tis true.
  MENTEITH. What wouldst thou?
  GANYMEDE. I did so much to save Edward
    As thou didst thyself.
  MENTEITH. O, thou didst me not?
  GANYMEDE. I am thy attorney, and thou wilt be
    My friend.
  MENTEITH. Too good for thee.
  GANYMEDE. Thou wilt, indeed, if I be thy friend.
  MENTEITH. Thou didst me well: I did well to save thee.
  GANYMEDE. I am thy friend.
  MENTEITH. If thou dost, thou wert not well.
  GANYMEDE. I do not think so.
  MENTEITH. Come, I pray you, come on; I know thee not.
  GANYMEDE. I think thou wert.
  MENTEITH. What, say I so?
  GANYMEDE. I do not know thou wert.
  MENTEITH. I know not thou wert.
  GANYMEDE. I know not, sir; but I think thou wert, sir,
    That I should be thy friend; and this is the ga

If you're creating an API based on your model and need to pass the generated text elsewhere, you can do `text = gpt2.generate(sess, return_as_list=True)[0]`

You can also pass in a `prefix` to the generate function to force the text to start with a given character sequence and generate text from there (good if you add an indicator when the text starts).

You can also generate multiple texts at a time by specifing `nsamples`. Unique to GPT-2, you can pass a `batch_size` to generate multiple samples in parallel, giving a massive speedup (in Colaboratory, set a maximum of 20 for `batch_size`).

Other optional-but-helpful parameters for `gpt2.generate` and friends:

*  **`length`**: Number of tokens to generate (default 1023, the maximum)
* **`temperature`**: The higher the temperature, the crazier the text (default 0.7, recommended to keep between 0.7 and 1.0)
* **`top_k`**: Limits the generated guesses to the top *k* guesses (default 0 which disables the behavior; if the generated output is super crazy, you may want to set `top_k=40`)
* **`top_p`**: Nucleus sampling: limits the generated guesses to a cumulative probability. (gets good results on a dataset with `top_p=0.9`)
* **`truncate`**: Truncates the input text until a given sequence, excluding that sequence (e.g. if `truncate='<|endoftext|>'`, the returned text will include everything before the first `<|endoftext|>`). It may be useful to combine this with a smaller `length` if the input texts are short.
*  **`include_prefix`**: If using `truncate` and `include_prefix=False`, the specified `prefix` will not be included in the returned text.

In [ ]:
gpt2.generate(sess,
              length=250,
              temperature=0.7,
              prefix="LORD",
              nsamples=5,
              batch_size=5
              )

LORD FITZWATER:
By heaven, I will not stand by while thy words are bandied about.

RICHARD:
Tyrannical times make good the sore; and
thy old quarrel oft again becomes a joy.

WARWICK:
So will I no longer stand by while he prove a fool.

KING RICHARD III:
Now Joan's a torch to keep on the king;
And his heir his mother's flame.

WARWICK:
Nay, my eyes will soon wither and die out;
For I will be the man, as the flower is the fruit.

KING RICHARD III:
The more fool you by so much, the more light you allow.

RICHARD:
I am too heavy a sleeper to rest my meaning;
But by the heavy fancy of my state
I have lighten'd the minstrels by soaring high.

KING RICHARD III:
Ay, but I bethink me of no other but this:
Come, let's away; the tediousness end.

RICHARD:
How now! what news with you?

LORD ROSS:
Yes, and by many, many
And many more, many, many;
And where I seem to stand, you may say,
My stand, or your bosom is changed,
My lord, my substitute, my sovereign.

CLIFFORD:
Then, good my lord,
Look to 

For bulk generation, you can generate a large amount of text to a file and sort out the samples locally on your computer. The next cell will generate a generated text file with a unique timestamp.

You can rerun the cells as many times as you want for even more generated texts!

In [ ]:
gen_file = 'gpt2_gentext_{:%Y%m%d_%H%M%S}.txt'.format(datetime.utcnow())

In [ ]:
gpt2.generate_to_file(sess,
                      destination_path=gen_file,
                      length=500,
                      temperature=0.7,
                      nsamples=100,
                      batch_size=20
                      )

NameError: name 'sess' is not defined

In [ ]:
# may have to run twice to get file to download
files.download(gen_file)

FileNotFoundError: Cannot find file: gpt2_gentext_20240618_132634.txt

## Generate Text From The Pretrained Model

If you want to generate text from the pretrained model, not a finetuned model, pass `model_name` to `gpt2.load_gpt2()` and `gpt2.generate()`.

This is currently the only way to generate text from the 774M or 1558M models with this notebook.

In [ ]:
model_name = "774M"

gpt2.download_gpt2(model_name=model_name)

Fetching checkpoint: 1.05Mit [00:00, 5.48Git/s]                                                     
Fetching encoder.json: 1.05Mit [00:00, 2.25Mit/s]
Fetching hparams.json: 1.05Mit [00:00, 4.15Git/s]                                                   
Fetching model.ckpt.data-00000-of-00001: 3.10Git [01:56, 26.6Mit/s]                                 
Fetching model.ckpt.index: 1.05Mit [00:00, 2.58Git/s]                                               
Fetching model.ckpt.meta: 2.10Mit [00:00, 3.58Mit/s]                                                
Fetching vocab.bpe: 1.05Mit [00:00, 3.21Mit/s]


In [ ]:
sess = gpt2.start_tf_sess()

gpt2.load_gpt2(sess, model_name=model_name)

Loading pretrained model models/774M/model.ckpt


In [ ]:
gpt2.generate(sess,
              model_name=model_name,
              prefix="The secret of life is",
              length=100,
              temperature=0.7,
              top_p=0.9,
              nsamples=5,
              batch_size=5
              )

The secret of life is that it is a mystery." - Charles Darwin

I had no idea what to expect when I entered the medical school. The only thing I knew was that I would be learning about medicine and how to cure cancer.

My senior year, I had a great chemistry teacher who was very passionate about cancer. He would write books on the subject. One of his books was on cancer and he'd explain to us how cancer grows and how we can find a cure. He was an amazing teacher
The secret of life is, in fact, not that it is good, but that it is useful, and that the more useful it is, the more intelligent it is. The more intelligent it is, the more it will learn to serve its own ends. The more useful it is, the more it will be capable of serving its own ends. The more useful it is, the more it will be capable of serving its own ends. The more useful it is, the more it will be capable of serving its own ends
The secret of life is in the depths of the universe. Life's foundation lies in the depths of spac

# Etcetera

If the notebook has errors (e.g. GPU Sync Fail), force-kill the Colaboratory virtual machine and restart it with the command below:

In [ ]:
!kill -9 -1

# LICENSE

MIT License

Copyright (c) 2019 Max Woolf

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.